In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Circle

from utils import Punto, PuntoCuerpo, Obj, Vector

# Importar datos
ruta = 'DataDLC/videos_eval/TS - Caja 1 - A_L - Position.h5'
df = pd.read_hdf(ruta)['DLC_resnet50_brainhackMay25shuffle1_230000']

# Crear puntos y objetos que vamos a usar
cabeza = PuntoCuerpo(df, 'head')
nariz = PuntoCuerpo(df, 'nose')
obj1 = Obj(df, 'obj_1')
obj2 = Obj(df, 'obj_2')

# Cantidad de cuadros
frames = len(cabeza.posiciones)

# Radios de los objetos
radio1 = np.array([5])
radio2 = 5

R1 = np.repeat(radio1, frames, axis = 0)

In [2]:
# Filtrar puntos cercanos al objeto 1
distanciaMaxima = 20 # cm
cercanos1 = nariz.posiciones[Punto.dist(nariz, obj1) < distanciaMaxima]

# Filtrar puntos cercanos al objeto 2
cercanos2 = nariz.posiciones[Punto.dist(nariz, obj2) < distanciaMaxima]

El criterio completo debería ser:

1. Nariz cercana al objeto.

2. Llamando $\theta$ al ángulo entre el vector cabeza-nariz y el vector cabeza-objeto, $d$ a la distancia entre la cabeza y el objeto y $R$ al radio del objeto (asumiendo que se trata de un círculo), el coseno del ángulo debería estar entre
$$\cos \theta_{mín} = \frac{d}{\sqrt{d² + R²}}$$
y $1$.

In [31]:
# Vector cabeza-objeto, normalizado
v0 = Vector(cabeza, obj1)

# Vector cabeza-nariz, normalizado
v1 = Vector(cabeza, nariz)

# Calcular el coseno del ángulo entre los vectores
cos = Vector.cosine(v0, v1)

# Filtrar puntos con cosenos dentro del rango
d = v0.norm

cosMin = d / np.sqrt(d**2 + R1**2)
mirando = nariz.posiciones[(cosMin <= cos) & (cos <= 1)]


(4048,)
(4048,)


### Graficar objetos y puntos cercanos

Falta adaptarlo para usar las clases.


```
fig, ax = plt.subplots()

ax.plot(xn, yn, ".", label = "Todos los puntos")


ax.plot(cercanos1[:, 0], cercanos1[:, 1], ".", label = "Cercanos a obj1", color = "orange")
ax.plot(cercanos2[:, 0], cercanos2[:, 1], ".", label = "Cercanos a obj2", color = "purple")

ax.plot([x1], [y1], "o", lw = 20, label = "Objeto 1", color = "red")
ax.plot([x2], [y2], "o", lw = 20, label = "Objeto 2", color = "blue")

ax.add_artist(Circle((x1, y1), distanciaMaxima, color = "grey", alpha = 0.3))
ax.add_artist(Circle((x2, y2), distanciaMaxima, color = "grey", alpha = 0.3))

ax.axis('equal')

ax.legend(bbox_to_anchor = (0, 0.2, 1, 1), ncol=3, fancybox=True, shadow=True)
```